In [2]:
# !pip install -qU python-gdcm pydicom pylibjpeg
# !pip install -U pylibjpeg-libjpeg -v
# !pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
# !pip install pydicom

# import include nesscessaryz
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

In [3]:
meta_rsna = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\rsna.csv')
meta_rsna= meta_rsna[['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]
# meta_rsna = meta_rsna[(meta_rsna.view == 'CC') |(meta_rsna.view == 'MLO') ]
meta_rsna

,patient_id,image_id,view,laterality,age,cancer
0,10006,462822612,CC,L,61.0,0
1,10006,1459541791,MLO,L,61.0,0
2,10006,1864590858,MLO,R,61.0,0
3,10006,1874946579,CC,R,61.0,0
4,10011,220375232,CC,L,55.0,0
...,...,...,...,...,...,...
54701,9973,1729524723,MLO,R,43.0,0
54702,9989,63473691,MLO,L,60.0,0
54703,9989,1078943060,CC,L,60.0,0
54704,9989,398038886,MLO,R,60.0,0


In [4]:
# Nhóm 2 cột id_img và cột cancer
check_cancer = {}
meta_rsna['id_img_zip'] = None
for i in range(len(meta_rsna)):
    # print(meta_rsna.image_id[i])
    check_cancer[meta_rsna.image_id[i]] = meta_rsna.cancer[i]
    meta_rsna['id_img_zip'][i] = f'{meta_rsna.cancer[i]}_{meta_rsna.image_id[i]}'
    

C:\Users\nguye\AppData\Local\Temp\ipykernel_13360\3275777359.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_rsna['id_img_zip'][i] = f'{meta_rsna.cancer[i]}_{meta_rsna.image_id[i]}'


In [5]:
meta_rsna[meta_rsna.cancer == 1]

,patient_id,image_id,view,laterality,age,cancer,id_img_zip
87,10130,388811999,MLO,L,71.0,1,1_388811999
88,10130,613462606,CC,L,71.0,1,1_613462606
89,10130,1360338805,CC,L,71.0,1,1_1360338805
90,10130,1672636630,MLO,L,71.0,1,1_1672636630
190,10226,461614796,MLO,L,71.0,1,1_461614796
...,...,...,...,...,...,...,...
54485,9707,1900919592,MLO,R,56.0,1,1_1900919592
54593,9840,111786935,MLO,R,59.0,1,1_111786935
54594,9840,1559360810,CC,R,59.0,1,1_1559360810
54595,9851,1409270544,MLO,L,58.0,1,1_1409270544


In [6]:
meta_rsna = meta_rsna[(meta_rsna.view == 'CC') |(meta_rsna.view == 'MLO')]

In [7]:
meta_rsna.view.value_counts()

MLO    27903
CC     26765
Name: view, dtype: int64

In [8]:
group_rsna = meta_rsna.groupby(['patient_id', 'view', 'laterality', 'id_img_zip', 'age']).size()

In [9]:
group_rsna

patient_id  view  laterality  id_img_zip    age 
5           CC    L           0_1351088028  68.0    1
                  R           0_940388076   68.0    1
            MLO   L           0_640805896   68.0    1
                  R           0_1633417959  68.0    1
25          CC    L           0_1789648218  62.0    1
                                                   ..
65530       MLO   R           0_675828643   48.0    1
65534       CC    L           0_1436834199  46.0    1
                  R           0_1174089649  46.0    1
            MLO   L           0_1888933323  46.0    1
                  R           0_44480502    46.0    1
Length: 54631, dtype: int64

In [10]:
len(list(group_rsna[25]['CC']['R'].index))

2

In [11]:
## Tạo một dataframe mới lưu thông tin ảnh đã xử lý
meta_pro = pd.DataFrame(columns=['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer'])
meta_pro
# data.loc[3] = ['PineApple','Yellow','48']

,patient_id,image_id,view,laterality,age,cancer


In [12]:
global id 
id = 0

In [13]:

def filter_for1x(par, views, lat, i):
    global id 
    A = list(group_rsna[par][views][lat].index)
    dic = {'CC': 'MLO', 'MLO': 'CC'}
    while i < len(A):   
        print(A[i][0].split('_'))
        if A[i][0].split('_')[0] == '1':
            print("YES")
            a, age_a = A[i]
            cancer_a, a = a.split('_')
            print(a)
            # cancer_a, a = a.split('_')
        
            ## update data
            # print(f'dict: {dic[lat]}')
            meta_pro.loc[id] = [par, a, views, lat, age_a, int(cancer_a)]
            meta_pro.loc[id+1] = [par, f'fake_{a}', dic[views], lat, age_a, 0]
            id+=2
        i+=1

In [14]:
ls = list(group_rsna[25]['CC']['R'].index)
ls = sorted(ls, key=lambda x: x[0])# ls[0][0]
ls

[('0_1997933901', 62.0), ('0_822390278', 62.0)]

In [15]:
def filter_for_2x(par, lat):
    # if len(group_rsna[par]['CC'][lat].index) != len(group_rsna[par]['MLO'][lat].index):
    global id 
    A = list(group_rsna[par]['CC'][lat].index)
    B = list(group_rsna[par]['MLO'][lat].index)
    A = sorted(A, key=lambda x: x[0],  reverse=True)# ls[0][0]
    B = sorted(B, key=lambda x: x[0],  reverse=True)# ls[0][0]
    i = 0
    j = 0
    print(A)
    print(B)
    while i < len(A) and j < len(B):
        a, age_a = A[i]
        b, age_b = B[i]
        cancer_a, a = a.split('_')
        cancer_b, b = b.split('_')
        ## update data
        meta_pro.loc[id] = [par, a, 'CC', lat, age_a, int(cancer_a)]
        
        meta_pro.loc[id+1] = [par, b, 'MLO', lat, age_b, int(cancer_b)]
        id+=2
        i+=1
        j+=1
        ## Kiem tra luong thua
    print(i, j)
    filter_for1x(par, 'CC', lat, i)
    filter_for1x(par, 'MLO', lat, j)

In [16]:
test1 = 54084
print(filter_for_2x(test1, 'R'))

[('1_852493177', 75.0), ('1_192075335', 75.0)]
[('1_1440235181', 75.0)]
1 1
['1', '852493177']
YES
852493177
None


In [17]:
meta_pro

,patient_id,image_id,view,laterality,age,cancer
0,54084,852493177,CC,R,75.0,1
1,54084,1440235181,MLO,R,75.0,1
2,54084,852493177,CC,R,75.0,1
3,54084,fake_852493177,MLO,R,75.0,0


In [18]:
meta_pro

,patient_id,image_id,view,laterality,age,cancer
0,54084,852493177,CC,R,75.0,1
1,54084,1440235181,MLO,R,75.0,1
2,54084,852493177,CC,R,75.0,1
3,54084,fake_852493177,MLO,R,75.0,0


In [19]:
def filter_main(par, lat):
    if check_cancer(par, 'CC', lat) and check_cancer(par, 'MLO', lat):
           filter_for_2x(par, lat=lat)
    elif  check_cancer(par, 'CC', lat) :
        
        filter_for1x(par, 'CC', lat=lat, i=0)
        
    elif  check_cancer(par, 'MLO', lat) :
        
        filter_for1x(par, 'MLO', lat=lat, i=0)

In [20]:

def check_cancer(par, views, lat):
    try: 
        # print(len(group_rsna[par][views][lat].index))
        return len(group_rsna[par][views][lat].index) >0
    except:
        return False
storage = []

def procee(par):
    ## Process img R
    
    filter_main(par, lat='R')

    ## Process img L
    filter_main(par, lat='L')
            
                    


    

In [21]:
# Nhóm các nhóm lại  dựa trên số img để thử nghiệm
# ['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']
level_par = meta_rsna.groupby(['patient_id']).size()
level_par = level_par.to_frame()
level_par = level_par.rename(columns={0:'Number_img'})
level_par.value_counts()

Number_img
4             8245
5             1769
6             1208
7              277
8              225
9               98
10              53
11              20
12              12
13               4
14               2
dtype: int64

In [22]:
level_5 = level_par[level_par.Number_img == 5].reset_index()
level_6 = level_par[level_par.Number_img == 6].reset_index()
level_7 = level_par[level_par.Number_img == 7].reset_index()

In [23]:
level_par.index

Int64Index([    5,    25,    28,    30,    33,    36,    45,    49,    65,
               71,
            ...
            65487, 65491, 65492, 65493, 65495, 65518, 65525, 65526, 65530,
            65534],
           dtype='int64', name='patient_id', length=11913)

In [24]:
for par in level_par.index:
    procee(par)

[('0_940388076', 68.0)]
[('0_1633417959', 68.0)]
1 1
[('0_1351088028', 68.0)]
[('0_640805896', 68.0)]
1 1
[('0_822390278', 62.0), ('0_1997933901', 62.0)]
[('0_1962456803', 62.0), ('0_1723979573', 62.0)]
2 2
[('0_1789648218', 62.0)]
[('0_1442180348', 62.0)]
1 1
[('0_2108025467', 40.0)]
[('0_145056531', 40.0)]
1 1
[('0_825270681', 40.0)]
[('0_1113093954', 40.0)]
1 1
[('0_244873631', 54.0)]
[('0_924301348', 54.0)]
1 1
[('0_1153448462', 54.0)]
[('0_275846607', 54.0)]
1 1
[('0_567858385', 43.0), ('0_1863578666', 43.0)]
[('0_1579089732', 43.0)]
1 1
['0', '567858385']
[('0_995020214', 43.0)]
[('0_2057464302', 43.0)]
1 1
[('0_200779059', 61.0)]
[('0_1161539227', 61.0)]
1 1
[('0_1877920136', 61.0)]
[('0_2109485229', 61.0), ('0_164115176', 61.0)]
1 1
['0', '2109485229']
[('0_1476454372', 57.0)]
[('0_2102838312', 57.0)]
1 1
[('0_310190625', 57.0)]
[('0_1476755745', 57.0)]
1 1
[('0_323049527', 53.0)]
[('0_872677361', 53.0), ('0_1074487659', 53.0)]
1 1
['0', '872677361']
[('0_373605482', 53.0)]
[('

KeyboardInterrupt: 

In [40]:
def save_file(df, name = "rsna"):
    df.to_csv(f'{name}.csv')

In [41]:
save_file(meta_pro, 'rsna_4')

In [38]:
meta_pro = pd.DataFrame(columns=['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer'])
meta_pro
for i in meta_rsna.patient_id.unique():
    procee(i)

[('0_1874946579', 61.0)]
[('0_1864590858', 61.0)]
1 1
[('0_462822612', 61.0)]
[('0_1459541791', 61.0)]
1 1
[('0_541722628', 55.0)]
[('0_1031443799', 55.0)]
1 1
[('0_220375232', 55.0)]
[('0_270344397', 55.0)]
1 1
[('0_288394860', 75.0)]
[('0_893612858', 75.0)]
1 1
[('0_562340703', 75.0)]
[('0_1365269360', 75.0)]
1 1
[('0_1350492010', 60.0)]
[('0_850559196', 60.0)]
1 1
[('0_2142944869', 60.0)]
[('0_1967300488', 60.0)]
1 1
[('0_294481519', 51.0)]
[('0_495770405', 51.0)]
1 1
[('0_102733848', 51.0)]
[('0_202939811', 51.0), ('0_1648588715', 51.0)]
1 1
['0', '202939811']
[('0_1842203124', 62.0)]
[('0_1577142909', 62.0)]
1 1
[('0_1234933874', 62.0)]
[('0_964141995', 62.0)]
1 1
[('0_1351270472', 52.0)]
[('0_94335194', 52.0), ('0_1207499426', 52.0)]
1 1
['0', '94335194']
[('0_349510516', 52.0)]
[('0_906829939', 52.0), ('0_1464813991', 52.0)]
1 1
['0', '906829939']
[('0_1614607569', 67.0)]
[('0_1428987847', 67.0)]
1 1
[('0_1749389520', 67.0)]
[('0_588678397', 67.0)]
1 1
[('0_2046475482', 53.0)]
[

## Phân tích metapro

In [43]:
meta_pro.view.value_counts()

CC     25088
MLO    25088
Name: view, dtype: int64

### Test code

In [27]:
# Test cancer

test_cancer = meta_rsna[meta_rsna.cancer == 1]
test_cancer

,patient_id,image_id,view,laterality,age,cancer,id_img_zip
87,10130,388811999,MLO,L,71.0,1,1_388811999
88,10130,613462606,CC,L,71.0,1,1_613462606
89,10130,1360338805,CC,L,71.0,1,1_1360338805
90,10130,1672636630,MLO,L,71.0,1,1_1672636630
190,10226,461614796,MLO,L,71.0,1,1_461614796
...,...,...,...,...,...,...,...
54485,9707,1900919592,MLO,R,56.0,1,1_1900919592
54593,9840,111786935,MLO,R,59.0,1,1_111786935
54594,9840,1559360810,CC,R,59.0,1,1_1559360810
54595,9851,1409270544,MLO,L,58.0,1,1_1409270544


In [28]:
level_par_cancer = test_cancer.groupby(['patient_id']).size()
level_par_cancer = level_par_cancer.to_frame()
level_par_cancer = level_par_cancer.rename(columns={0:'Number_img'})
level_par_cancer.value_counts()

Number_img
2             343
3             108
4              29
5               6
dtype: int64

In [29]:
level_par_cancer = level_par_cancer.reset_index()

In [37]:
meta_rsna.patient_id.unique()

array([10006, 10011, 10025, ...,   997,  9973,  9989], dtype=int64)

In [30]:
for i in meta_rsna.patient_id.unique():
    procee(i)

[('0_1160585918', 65.0)]
[('0_1495797827', 65.0)]
1 1
[('1_76321767', 65.0)]
[('1_2018825992', 65.0)]
1 1
[('0_435616956', 68.0)]
[('0_1600147897', 68.0), ('0_1563922629', 68.0)]
1 1
['0', '1600147897']
[('1_1593856707', 68.0)]
[('1_1531879119', 68.0)]
1 1
[('1_352547246', 53.0)]
[('1_1850429460', 53.0)]
1 1
[('0_494974784', 53.0)]
[('0_2019013000', 53.0)]
1 1
[('0_1083423452', 72.0)]
[('0_1732744845', 72.0)]
1 1
[('1_1942815129', 72.0)]
[('1_1923583324', 72.0)]
1 1
[('1_730395278', 63.0)]
[('1_748167281', 63.0)]
1 1
[('0_537917766', 63.0)]
[('0_191513889', 63.0)]
1 1
[('0_668064372', 68.0)]
[('0_2056634184', 68.0)]
1 1
[('1_981350543', 68.0)]
[('1_343058323', 68.0)]
1 1
[('1_2075900290', 55.0)]
[('1_806916891', 55.0)]
1 1
[('0_631715434', 55.0)]
[('0_1513027022', 55.0)]
1 1
[('0_347069540', 56.0)]
[('0_1122776493', 56.0)]
1 1
[('1_773597682', 56.0)]
[('1_1803952236', 56.0)]
1 1
[('1_1326732476', 57.0)]
[('1_1869230850', 57.0)]
1 1
[('0_933056599', 57.0)]
[('0_1424257315', 57.0), ('0_1

In [238]:
meta_pro.reset_index().to_csv('test1.csv')

In [31]:
import random

In [32]:
def show(id):
    print(meta_pro[meta_pro.patient_id == id])
    print(meta_rsna[meta_rsna.patient_id == id])
ls = list(level_par_cancer.patient_id.values)


In [33]:
cnt_test = 3

In [34]:
# test1 = random.choice(ls)
test1 = 10589
print(test1)
# print(ls)
# for i in test1:
print(f'Start {test1} ------------')
show(test1)
cnt_test+=1
# ls

10589
Start 10589 ------------
      patient_id        image_id view laterality   age  cancer
5930       10589       363277205   CC          R  74.0       0
5931       10589      2059883725  MLO          R  74.0       0
5932       10589      1967460233   CC          L  74.0       1
5933       10589       605115808  MLO          L  74.0       1
5934       10589       605115808  MLO          L  74.0       1
5935       10589  fake_605115808   CC          L  74.0       0
     patient_id    image_id view laterality   age  cancer    id_img_zip
569       10589   195400299  MLO          L  74.0       1   1_195400299
570       10589   605115808  MLO          L  74.0       1   1_605115808
571       10589  1967460233   CC          L  74.0       1  1_1967460233
572       10589   363277205   CC          R  74.0       0   0_363277205
573       10589  2059883725  MLO          R  74.0       0  0_2059883725


In [372]:
cnt_test

16

## Had CHECK 16 times return OK